<a href="https://colab.research.google.com/github/afit-csce623-master/demos/blob/main/demo_ridge_lasso_elasticnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This tutorial developed by W. Blair Watkinson II
# Apr 30, 2021

# Developed in support of education and research activities at 
# the Air Force Institute of Technology and the CSCE 623
# Machine Learning course

# For changes or questions, contact warren.watkinson@afit.edu

# Public Domain

# This is free and unencumbered software released into the public domain.

# Anyone is free to copy, modify, publish, use, compile, sell, or
# distribute this software, either in source code form or as a compiled
# binary, for any purpose, commercial or non-commercial, and by any
# means.

# In jurisdictions that recognize copyright laws, the author or authors
# of this software dedicate any and all copyright interest in the
# software to the public domain. We make this dedication for the benefit
# of the public at large and to the detriment of our heirs and
# successors. We intend this dedication to be an overt act of
# relinquishment in perpetuity of all present and future rights to this
# software under copyright law.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
# IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR
# OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
# ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
# OTHER DEALINGS IN THE SOFTWARE.

# For more information, please refer to <https://unlicense.org>

# Ridge, Lasso, and ElasticNet Demo

This notebook demonstrates the use of Ridge, Lasso, and ElasticNet on a linear regression dataset.

## Generate Data

Here we generate regression data with 8 features, but only 3 features are needed to determine the target `y`.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
import time

from IPython.display import Markdown as md
from IPython.display import display, Math, Latex

# initialize random state and regression parameters
random_state = np.random.RandomState(101)
n_features = 8
n_informative = 3
bias_data = random_state.uniform(0,10)
noise = 2

# Build a regression task
X, y, coef_data = make_regression(n_samples=6000, 
                                  n_features=n_features, n_informative=n_informative, 
                                  bias=bias_data, noise=noise,
                                  random_state=random_state, coef=True)

## Helper Functions

The following helper functions will be used throughout this tutorial.

### Generate Model Function

This function generates a model string of the form $y = \beta_0 + \beta_1 x_1 + \dots + \beta_n x_n$ where $n$ is the number of coefficients in `betas`.

In [2]:
def generate_model_function(beta0, betas, feature_names):
    if np.isclose(beta0, 0):
        model_function = f'$y = '
    else:
        model_function = f'$y = {beta0:.3f} + '

    for beta, feature_name in zip(betas, feature_names):
        model_function += f'{beta:.3f}{feature_name} + '
    
    return model_function[:-3] + '$'

### Clean Up Model

Some models don't clean-up very small coefficients, leaving values looking like $ \dots + 0.000 x_3 + \dots $. This function will filter out coefficients that are near zero.

In [3]:
def clean_up_model(coefs, names):
    non_zero_coef_filter = np.logical_not(np.isclose(coefs, 0))
    non_zero_coefs = coefs[non_zero_coef_filter]
    non_zero_names = names[non_zero_coef_filter]

    return non_zero_coefs, non_zero_names


### Generate Clean Model

This function combines the above two functions to return a cleaned-up model string.

In [4]:
def generate_clean_model(beta0, coefs, names):
    coefs_, names_ = clean_up_model(coefs, names)
    return generate_model_function(beta0, coefs_, names_)

### Powerset Function

This function generates the power set of a list, excluding the empty set.

In [5]:
# powerset function

from itertools import chain, combinations

def powerset(iterable, j=1, k=999):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)

    # start with 1, as we don't need the empty set
    p = list(chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1)))
    return [item for item in p if len(item) >= j and len(item) <= k]

feature_powerset = powerset(range(n_features))

### Time String

In [6]:
def convert_to_time(seconds):
    return time.strftime('%H:%M:%S', time.gmtime(seconds))

### Progress String

In [7]:
def print_progress_str(start_time, trial_time, total_trials, current_trial_idx, feature_set_name_str, best_score, current_score):

    time_elapsed_str = convert_to_time(time.perf_counter() - start_time)
    time_trial_sec = time.perf_counter() - trial_time

    status_str = f'{time_elapsed_str} {time_trial_sec:.2f}s {current_trial_idx} of {total_trials}: {feature_set_name_str} {current_score}'
    remaining_time = (time.perf_counter() - start_time) / current_trial_idx * (total_trials - current_trial_idx)
    
    if (current_score < best_score):
        print('\r', status_str)
    else:
        print('\r', status_str, f'Time Remaining: {time.strftime("%H:%M:%S", time.gmtime(remaining_time))}', end='')

## Prepatory actons

Here, we split the train and test datasets

In [8]:
from sklearn.model_selection import train_test_split

# split and sequester test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

We set some variables that we will reuse throught the notebook.

In [9]:
kfold_size = 5

# create feature names -- these are generic feature names x_1, x_2, ..., x_25
feature_names = np.array([f'x_{{{idx+1}}}' for idx in range(n_features)])

## Ridge

### Set alpha values

We will set alpha values. With Ridge regression, we will include 0.



In [10]:
alpha_min = 0.0000001
alpha_max = 100000
alpha_num = int(abs(np.log10(alpha_min)) + abs(np.log10(alpha_max)) + 1)
alphas = np.geomspace(alpha_min, alpha_max, num=alpha_num)
alphas = np.insert(alphas, 0, 0)

### Ridge - manual cross validation

The following code uses the Ridge regression model for each alpha value. We keep track of each RMSE, and select the model that yielded the lowest RMSE. We compare the model with the model that generated the original dataset.

In [11]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

df_ridge = pd.DataFrame(columns=['alpha', 'rmse', 'r2'])

for alpha in alphas:
    ridge = Ridge(alpha=alpha, fit_intercept=True, random_state=random_state)
    scores = cross_val_score(ridge, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfold_size)
    ridge.fit(X_train, y_train)
    df_ridge.loc[len(df_ridge.index)] = [alpha, -np.mean(scores), ridge.score(X_test, y_test)]

best_ridge = {}
best_ridge['alpha'] = df_ridge.loc[np.argmin(df_ridge['rmse'])]['alpha']
best_ridge['model'] = Ridge(alpha=best_ridge['alpha'], fit_intercept=True, random_state=random_state)
best_ridge['model'].fit(X_train, y_train)
best_ridge['r2'] = best_ridge['model'].score(X_test, y_test)
y_pred = best_ridge['model'].predict(X_test)
best_ridge['rmse'] = mean_squared_error(y_test, y_pred, squared=False)

display(md(f'Best Ridge with $\\alpha$={best_ridge["alpha"]}: ' +
           f'{(generate_clean_model(best_ridge["model"].intercept_, best_ridge["model"].coef_, feature_names))}, ' +
           f'RMSE: {best_ridge["rmse"]:.4f}, $R^2$: {best_ridge["r2"]:.4f}'))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

Best Ridge with $\alpha$=0.0: $y = 5.190 + 0.004x_{1} + 39.521x_{2} + -0.001x_{3} + 16.122x_{4} + 34.609x_{5} + 0.015x_{6} + -0.001x_{7} + -0.023x_{8}$, RMSE: 2.0544, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

### RidgeCV

The following code uses the RidgeCV to automatically find the alpha value that yields the smallest RMSE. We compare the model with the model that generated the original dataset.

In [12]:
from sklearn.linear_model import RidgeCV

ridgeCV = RidgeCV(alphas=alphas,
                  scoring='neg_root_mean_squared_error',
                  cv=kfold_size)
ridgeCV.fit(X_train, y_train)
ridgeCV_r2 = ridgeCV.score(X_test, y_test)
y_pred = ridgeCV.predict(X_test)
ridgeCV_rmse = mean_squared_error(y_test, y_pred, squared=False)
display(md(f'RidgeCV with $\\alpha$={ridgeCV.alpha_}: ' +
           f'{(generate_clean_model(ridgeCV.intercept_, ridgeCV.coef_, feature_names))}, ' +
           f'RMSE: {ridgeCV_rmse:.4f}, $R^2$: {ridgeCV_r2:.4f}'))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

RidgeCV with $\alpha$=0.0: $y = 5.190 + 0.004x_{1} + 39.521x_{2} + -0.001x_{3} + 16.122x_{4} + 34.609x_{5} + 0.015x_{6} + -0.001x_{7} + -0.023x_{8}$, RMSE: 2.0544, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

### Ridge with subsets of features

Neither Ridge nor RidgeCV is able to fully eliminate unimportant features. Here, we test the power set of features to find the set of features and alpha value that generates the lowest RMSE.

In [13]:
# use power set

df_ridge = pd.DataFrame(columns=['feature_set', 'alpha', 'rmse']) #, 'r2'])

trial_time = time.perf_counter()
start_time = time.perf_counter()
total_trials = len(feature_powerset) * len(alphas)
current_trial_idx = 0
best_rmse = 1000

for idx_feature, feature_set in enumerate(feature_powerset):

    feature_set_str = ','.join(str(e) for e in feature_set) # current feature set as string
    feature_set_name_str = ','.join(str(e) for e in feature_names[np.array(feature_set)]) # feature set names as string

    for idx_alpha, alpha in enumerate(alphas):
        current_trial_idx += 1

        ridge = Ridge(alpha=alpha, fit_intercept=True, random_state=random_state)
        scores = cross_val_score(ridge, X_train[:,np.array(feature_set)], y_train, scoring='neg_root_mean_squared_error', cv=kfold_size)

        # store details in dataframe for later accessing
        df_ridge.loc[len(df_ridge.index)] = [feature_set_str, alpha, -np.mean(scores)]

        # print to show progress
        print_progress_str(start_time, trial_time, total_trials, current_trial_idx, feature_set_name_str, best_rmse, -np.mean(scores))

        # update best_rmse
        best_rmse = -np.mean(scores) if -np.mean(scores) < best_rmse else best_rmse

        trial_time = time.perf_counter()

print('\r', f'Total time: {convert_to_time(time.perf_counter() - start_time)}')

best_ridge = {}
best_ridge['params'] = df_ridge.loc[np.argmin(df_ridge['rmse'])]
best_ridge['features'] = [int(e) for e in best_ridge['params']['feature_set'].split(',')]
best_ridge['feature_names'] = feature_names[np.array(best_ridge['features'])]
best_ridge['alpha'] = best_ridge['params']['alpha']

best_ridge['model'] = Ridge(alpha=best_ridge['alpha'])
best_ridge['model'].fit(X_train[:,np.array(best_ridge['features'])], y_train)
best_ridge['r2'] = best_ridge['model'].score(X_test[:,np.array(best_ridge['features'])], y_test)
y_pred = best_ridge['model'].predict(X_test[:,np.array(best_ridge['features'])])
best_ridge['rmse'] = mean_squared_error(y_test, y_pred, squared=False)

display(md(f'Best Ridge with $\\alpha$={best_ridge["alpha"]}: ' +
           f'{(generate_clean_model(best_ridge["model"].intercept_, best_ridge["model"].coef_, best_ridge["feature_names"]))}, ' + 
           f'RMSE: {best_ridge["rmse"]:.4f}, ' +
           f'$R^2$: {best_ridge["r2"]:.4f}'
           ))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

 00:00:00 0.02s 0 of 3570: x_{1} 54.94646561692082
 00:00:00 0.02s 0 of 3570: x_{1} 54.94646561692032
 00:00:00 0.02s 0 of 3570: x_{1} 54.94646561691578
 00:00:00 0.02s 0 of 3570: x_{1} 54.946465616870434
 00:00:00 0.01s 0 of 3570: x_{1} 54.946465616417015
 00:00:00 0.02s 0 of 3570: x_{1} 54.94646561188281
 00:00:00 0.01s 0 of 3570: x_{1} 54.946465566540816
 00:00:00 0.01s 0 of 3570: x_{1} 54.94646511313441
 00:00:00 0.02s 0 of 3570: x_{1} 54.94646058042177
 00:00:00 0.01s 0 of 3570: x_{1} 54.946415388038226
 00:00:00 0.01s 0 of 3570: x_{1} 54.945976550772755
 00:00:00 0.01s 0 of 3570: x_{1} 54.94259711361771
 00:00:00 0.01s 0 of 3570: x_{1} 54.93404193564304
 00:00:00 0.01s 0 of 3570: x_{1} 54.93058194410062
 00:00:00 0.02s 1 of 3570: x_{2} 38.54805627120733
 00:00:03 0.02s 15 of 3570: x_{2},x_{3} 38.54774111644366
 00:00:03 0.02s 15 of 3570: x_{2},x_{3} 38.547741116443504
 00:00:03 0.02s 15 of 3570: x_{2},x_{3} 38.5477411164421
 00:00:03 0.02s 15 of 3570: x_{2},x_{3} 38.5477411164280

Best Ridge with $\alpha$=0.0: $y = 5.190 + 39.521x_{2} + 16.122x_{4} + 34.609x_{5}$, RMSE: 2.0540, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

## Lasso

### Set alpha values

We will set alpha values. With Lasso regression, we will not include 0.

In [14]:
alpha_min = 0.0000001
alpha_max = 100000
alpha_num = int(abs(np.log10(alpha_min)) + abs(np.log10(alpha_max)) + 1)
alphas = np.geomspace(alpha_min, alpha_max, num=alpha_num)

# Lasso is unreliable with alpha=0, use LinearRegression instead when alpha=0 is desired
#alphas = np.insert(alphas, 0, 0)

### Lasso - manual cross validation

The following code uses the Lasso regression model for each alpha value. We keep track of each RMSE, and select the model that yielded the lowest RMSE. We compare the model with the model that generated the original dataset.

In [15]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

df_lasso = pd.DataFrame(columns=['alpha', 'rmse', 'r2'])

for alpha in alphas:
    lasso = Lasso(alpha=alpha, fit_intercept=True, random_state=random_state)
    scores = cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfold_size)
    lasso.fit(X_train, y_train)
    df_lasso.loc[len(df_lasso.index)] = [alpha, -np.mean(scores), lasso.score(X_test, y_test)]

best_lasso = {}
best_lasso['alpha'] = df_lasso.loc[np.argmin(df_lasso['rmse'])]['alpha']
best_lasso['model'] = Lasso(alpha=best_lasso['alpha'], fit_intercept=True, random_state=random_state)
best_lasso['model'].fit(X_train, y_train)
best_lasso['r2'] = best_lasso['model'].score(X_test, y_test)
y_pred = best_lasso['model'].predict(X_test)
best_lasso['rmse'] = mean_squared_error(y_test, y_pred, squared=False)

display(md(f'Best Ridge with $\\alpha$={best_lasso["alpha"]}: ' +
           f'{(generate_clean_model(best_lasso["model"].intercept_, best_lasso["model"].coef_, feature_names))}, ' +
           f'RMSE: {best_lasso["rmse"]:.4f}, $R^2$: {best_lasso["r2"]:.4f}'))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

Best Ridge with $\alpha$=0.01: $y = 5.190 + 39.511x_{2} + 16.112x_{4} + 34.599x_{5} + 0.005x_{6} + -0.012x_{8}$, RMSE: 2.0544, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

### LassoCV

The following code uses the LassoCV to automatically find the alpha value that yields the smallest RMSE. We compare the model with the model that generated the original dataset.

In [16]:
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(alphas=alphas,
                #   scoring='neg_root_mean_squared_error',
                  cv=kfold_size)
lassoCV.fit(X_train, y_train)
lassoCV_r2 = lassoCV.score(X_test, y_test)
y_pred = lassoCV.predict(X_test)
lassoCV_rmse = mean_squared_error(y_test, y_pred, squared=False)
display(md(f'LassoCV with $\\alpha$={lassoCV.alpha_}: ' +
           f'{(generate_clean_model(lassoCV.intercept_, lassoCV.coef_, feature_names))}, ' +
           f'RMSE: {lassoCV_rmse:.4f}, $R^2$: {lassoCV_r2:.4f}'))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

LassoCV with $\alpha$=0.01: $y = 5.190 + 39.511x_{2} + 16.112x_{4} + 34.599x_{5} + 0.005x_{6} + -0.012x_{8}$, RMSE: 2.0544, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

### Lasso with subsets of features

Though it performs better than Ridge, neither Lasso nor LassoCV is able to fully eliminate unimportant features. Here, we test the power set of features to find the set of features and alpha value that generates the lowest RMSE.

In [19]:
# use power set

df_lasso = pd.DataFrame(columns=['feature_set', 'alpha', 'rmse'])

trial_time = time.perf_counter()
start_time = time.perf_counter()
total_trials = len(feature_powerset) * len(alphas)
current_trial_idx = 0
best_rmse = 1000

for feature_set in feature_powerset:

    feature_set_str = ','.join(str(e) for e in feature_set) # current feature set as string
    feature_set_name_str = ','.join(str(e) for e in feature_names[np.array(feature_set)]) # feature set names as string

    for alpha in alphas:
        current_trial_idx += 1

        lasso = Lasso(alpha=alpha, fit_intercept=True, random_state=random_state)
        scores = cross_val_score(lasso, X_train[:,np.array(feature_set)], y_train, scoring='neg_root_mean_squared_error', cv=kfold_size)

        # store details in dataframe for later accessing
        df_lasso.loc[len(df_lasso.index)] = [feature_set_str, alpha, -np.mean(scores)]

        # print to show progress
        print_progress_str(start_time, trial_time, total_trials, current_trial_idx, feature_set_name_str, best_rmse, -np.mean(scores))

        # update best_rmse
        best_rmse = -np.mean(scores) if -np.mean(scores) < best_rmse else best_rmse

        trial_time = time.perf_counter()

print('\r', f'Total time: {convert_to_time(time.perf_counter() - start_time)}')

best_lasso = {}
best_lasso['params'] = df_lasso.loc[np.argmin(df_lasso['rmse'])]
best_lasso['features'] = [int(e) for e in best_lasso['params']['feature_set'].split(',')]
best_lasso['feature_names'] = feature_names[np.array(best_lasso['features'])]
best_lasso['alpha'] = best_lasso['params']['alpha']

best_lasso['model'] = Lasso(alpha=best_lasso['alpha'])
best_lasso['model'].fit(X_train[:,np.array(best_lasso['features'])], y_train)
best_lasso['r2'] = best_lasso['model'].score(X_test[:,np.array(best_lasso['features'])], y_test)
y_pred = best_lasso['model'].predict(X_test[:,np.array(best_lasso['features'])])
best_lasso['rmse'] = mean_squared_error(y_test, y_pred, squared=False)

display(md(f'Best Lasso with $\\alpha$={best_lasso["alpha"]}: ' +
           f'{(generate_clean_model(best_lasso["model"].intercept_, best_lasso["model"].coef_, best_lasso["feature_names"]))}, ' + 
           f'RMSE: {best_lasso["rmse"]:.4f}, ' +
           f'$R^2$: {best_lasso["r2"]:.4f}'
           ))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

 00:00:00 0.02s 254 of 3315: x_{1} 54.94646561320748
 00:00:00 0.01s 254 of 3315: x_{1} 54.94646557978742
 00:00:00 0.02s 254 of 3315: x_{1} 54.9464652455877
 00:00:00 0.01s 254 of 3315: x_{1} 54.94646190367321
 00:00:00 0.01s 254 of 3315: x_{1} 54.94642849280281
 00:00:00 0.02s 254 of 3315: x_{1} 54.946095211563076
 00:00:00 0.01s 254 of 3315: x_{1} 54.94284515107049
 00:00:00 0.02s 254 of 3315: x_{1} 54.9300791092254
 00:00:00 0.02s 254 of 3315: x_{2} 38.54805627128544
 00:00:03 0.02s 254 of 3315: x_{2},x_{3} 38.54774111642003
 00:00:03 0.02s 254 of 3315: x_{2},x_{3} 38.547741116268114
 00:00:03 0.02s 254 of 3315: x_{2},x_{3} 38.54774111381802
 00:00:03 0.02s 254 of 3315: x_{2},x_{3} 38.54774109526809
 00:00:03 0.02s 254 of 3315: x_{2},x_{3} 38.54774093409294
 00:00:03 0.01s 254 of 3315: x_{2},x_{4} 35.01764983856283
 00:00:03 0.02s 254 of 3315: x_{2},x_{5} 16.238228254325243
 00:00:16 0.02s 254 of 3315: x_{2},x_{4},x_{5} 1.95644959553184
 00:00:16 0.02s 254 of 3315: x_{2},x_{4},x_{5

Best Lasso with $\alpha$=1e-05: $y = 5.190 + 39.521x_{2} + 16.122x_{4} + 34.609x_{5}$, RMSE: 2.0540, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

## ElasticNet

### Set alpha values

We will set alpha values. With ElasticNet regression, which includes Lasso when l1_ratio=1, we will not include 0.

In [20]:
alpha_min = 0.0000001
alpha_max = 100000
alpha_num = int(abs(np.log10(alpha_min)) + abs(np.log10(alpha_max)) + 1)
alphas = np.geomspace(alpha_min, alpha_max, num=alpha_num)

# Lasso is unreliable with alpha=0, use LinearRegression instead when alpha=0 is desired
#alphas = np.insert(alphas, 0, 0)

### Set lr_ratio values

Set l1_ratio values. This is that amount of l1 regularization the linear model will attempt to perform (l1_ratio = 1.0 corresponds with Lasso, and l1_ratio = 0.0 corresponds with Ridge).

In [21]:
l1_ratios = np.linspace(0.01, 1.0 ,num=100)

### ElasticNetCV

The following code uses the ElasticNetCV to automatically find the alpha and l1_ratio values that yield the smallest RMSE. We compare the model with the model that generated the original dataset.

In [22]:
from sklearn.linear_model import ElasticNetCV

elastic_netCV = ElasticNetCV(alphas=alphas,
                      l1_ratio = l1_ratios,
                #   scoring='neg_root_mean_squared_error',
                  cv=kfold_size,
                  max_iter=1000)
elastic_netCV.fit(X_train, y_train)
elastic_netCV_r2 = elastic_netCV.score(X_test, y_test)
y_pred = elastic_netCV.predict(X_test)
elastic_netCV_rmse = mean_squared_error(y_test, y_pred, squared=False)
display(md(f'ElasticCV with $\\alpha$={elastic_netCV.alpha_}, $l1$ ratio={elastic_netCV.l1_ratio_}: ' +
           f'{(generate_clean_model(elastic_netCV.intercept_, elastic_netCV.coef_, feature_names))}, ' +
           f'RMSE: {elastic_netCV_rmse:.4f}, $R^2$: {elastic_netCV_r2:.4f}'))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

ElasticCV with $\alpha$=0.01, $l1$ ratio=1.0: $y = 5.190 + 39.511x_{2} + 16.112x_{4} + 34.599x_{5} + 0.005x_{6} + -0.012x_{8}$, RMSE: 2.0544, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$

### ElasticNet with subsets of features

ElasticNetCV's performance in this case is nearly identical to that of Lasso.Here, we test the power set of features to find the set of features, alpha, and l1_ratio values that generate the lowest RMSE.

In [23]:
# use power set

from sklearn.linear_model import ElasticNet

df_elastic_net = pd.DataFrame(columns=['feature_set', 'alpha', 'rmse'])

trial_time = time.perf_counter()
start_time = time.perf_counter()
total_trials = len(feature_powerset) * len(alphas) * len(l1_ratios)
current_trial_idx = 0
best_rmse = 1000

for feature_set in feature_powerset:

    feature_set_str = ','.join(str(e) for e in feature_set) # current feature set as string
    feature_set_name_str = ','.join(str(e) for e in feature_names[np.array(feature_set)]) # feature set names as string

    for alpha in alphas:
        for l1_ratio in l1_ratios:
            current_trial_idx += 1

            # create model
            elastic_net = ElasticNet(alpha=alpha, fit_intercept=True, random_state=random_state)

            # calculate rmse scores
            scores = cross_val_score(elastic_net, X_train[:,np.array(feature_set)], y_train, scoring='neg_root_mean_squared_error', cv=kfold_size)

            # store details in dataframe for later accessing
            df_elastic_net.loc[len(df_elastic_net.index)] = [feature_set_str, alpha, -np.mean(scores)]

            # print to show progress
            print_progress_str(start_time, trial_time, total_trials, current_trial_idx, feature_set_name_str, best_rmse, -np.mean(scores))

            # update best_rmse
            best_rmse = -np.mean(scores) if -np.mean(scores) < best_rmse else best_rmse

            trial_time = time.perf_counter()

print('\r', f'Total time: {convert_to_time(time.perf_counter() - start_time)}')

best_elastic_net = {}
best_elastic_net['params'] = df_elastic_net.loc[np.argmin(df_elastic_net['rmse'])]
best_elastic_net['features'] = [int(e) for e in best_elastic_net['params']['feature_set'].split(',')]
best_elastic_net['feature_names'] = feature_names[np.array(best_elastic_net['features'])]
best_elastic_net['alpha'] = best_elastic_net['params']['alpha']

best_elastic_net['model'] = ElasticNet(alpha=best_elastic_net['alpha'])
best_elastic_net['model'].fit(X_train[:,np.array(best_elastic_net['features'])], y_train)
best_elastic_net['r2'] = best_elastic_net['model'].score(X_test[:,np.array(best_elastic_net['features'])], y_test)
y_pred = best_elastic_net['model'].predict(X_test[:,np.array(best_elastic_net['features'])])
best_elastic_net['rmse'] = mean_squared_error(y_test, y_pred, squared=False)

display(md(f'Best ElasticNet with $\\alpha$={best_elastic_net["alpha"]}, $l1$ ratio={elastic_netCV.l1_ratio_}: ' +
           f'{(generate_clean_model(best_elastic_net["model"].intercept_, best_elastic_net["model"].coef_, best_elastic_net["feature_names"]))}, ' + 
           f'RMSE: {best_elastic_net["rmse"]:.4f}, ' +
           f'$R^2$: {best_elastic_net["r2"]:.4f}'
           ))
display(md(f'Original Data: {(generate_clean_model(bias_data, coef_data, feature_names))}'))

 00:00:00 0.02s 254 of 331500: x_{1} 54.94646561409685
 00:00:01 0.01s 254 of 331500: x_{1} 54.94646558868114
 00:00:03 0.01s 254 of 331500: x_{1} 54.94646533452584
 00:00:04 0.02s 254 of 331500: x_{1} 54.94646279314649
 00:00:06 0.01s 254 of 331500: x_{1} 54.94643739671174
 00:00:08 0.01s 254 of 331500: x_{1} 54.94618515717078
 00:00:09 0.01s 254 of 331500: x_{1} 54.94382481191995
 00:00:11 0.02s 254 of 331500: x_{1} 54.933119653278126
 00:00:13 0.01s 254 of 331500: x_{1} 54.9300791092254
 00:00:21 0.01s 254 of 331500: x_{2} 38.54805627222929
 00:05:33 0.01s 254 of 331500: x_{2},x_{3} 38.54774111613717
 00:05:35 0.02s 254 of 331500: x_{2},x_{3} 38.54774111338162
 00:05:37 0.02s 254 of 331500: x_{2},x_{3} 38.54774108545722
 00:05:38 0.01s 254 of 331500: x_{2},x_{3} 38.5477408620678
 00:05:56 0.02s 254 of 331500: x_{2},x_{4} 35.017649839097416
 00:06:19 0.02s 254 of 331500: x_{2},x_{5} 16.23822825406914
 00:06:21 0.02s 254 of 331500: x_{2},x_{5} 16.238228252019876
 00:06:23 0.02s 254 of

Best ElasticNet with $\alpha$=1e-07, $l1$ ratio=1.0: $y = 5.190 + 39.521x_{2} + 16.122x_{4} + 34.609x_{5}$, RMSE: 2.0540, $R^2$: 0.9987

Original Data: $y = 5.164 + 39.530x_{2} + 16.115x_{4} + 34.584x_{5}$